$$\require{mhchem}$$ 

# AP 275 Lab 2: Introduction Quantum Chemistry with PySCF


*Instructions*: Place this file in your Google Drive and open it with Google Colaborary. When finished, print the notebook and submit the PDF.

The exercises in this notebook will illustrate the basics of using the open-source PySCF package, which is a Gaussian-type orbital (GTO) code for quantum chemistry. Before getting started, we will review two important concepts that practitioners of quantum chemistry codes need to understand: the **basis set** and **level of theory** of a calculation. This introduction assumes that you are familiar with the material in the DFT lectures presented in class. 



## Basis Set

In order to solve the Schr&ouml;dinger equation, we need to express the electron wave functions $|\psi_i\rangle$ as a linear combination of basis functions $\chi_\mu(\mathbf{r})$:
\begin{equation}
\psi_i(\mathbf{r})=\sum_{\mu=1}^{N_{\text{bas}}} C_{\mu i} \chi_\mu(\mathbf{r})
\end{equation}
Then, $|\psi_i\rangle$ can be represented as a column vector of coefficients $\mathbf{c}_i=[C_{1,i}, ..., C_{\mu,i}, ... C_{N_{\text{bas},i}}]$. The set of functions $\{\chi_\mu(\mathbf{r})\}$ is the **basis set**, and the number of basis functions $N_{\text{bas}}$ is the **basis set size.**

The basis functions $\chi_\mu(\mathbf{r})$ can be whatever we want them to be, as long as they are linearly independent and normalizable. Therefore, the form of the basis functions is typically chosen using two criteria:
1. Represent the possible shapes of the wave functions as efficiently as possible (i.e. with as few basis functions as possible).
2. Use a mathematical form that is convenient and computationally efficient to manipulate mathematically.

For example, in calculations for periodic systems, plane-waves are typically used as the basis, i.e. $\chi_\mu(\mathbf{r})=e^{i\mathbf{G}_\mu\cdot\mathbf{r}}$. While many plane-waves are required to represent the wavefunction, the periodicity of the plane-waves is useful for solids. On top of that, plane-waves are orthogonal and lend themselves to the efficient use of Fourier transforms to construct the Hamiltonian. For isolated (non-periodic) systems, different basis sets are required, as discussed below.

### LCAO and Gaussian-Type Orbitals

**For isolated molecular systems**, it is more efficient to use *atomic orbitals* for $\chi_\mu(\mathbf{r})$ because they are localized in a small region of space around an atom, which is more conducive to representing the wave functions of an isolated molecule than periodic, delocalized plane-waves. Because $\psi_i(\mathbf{r})$ is a linear combination of the basis functions, this method is called the **linear combination of atomic orbitals (LCAO)**.

The most popular functions used for constructing atomic orbitals for the LCAO method are 3D Gaussians, which are 3D functions consisting of a spherical harmonic function of the angular coordinates multiplied by a Gaussian function of the radial coordinate:
\begin{equation}
g_{\alpha lm}(\mathbf{r}) = N_{\alpha lm} |\mathbf{r}|^l  Y_{lm}(\hat{\mathbf{r}})\exp\left(-\alpha |\mathbf{r}|^2\right)
\end{equation}
These functions, which are called **Gaussian-type orbitals (GTOs)**, look complicated, but they have a few properties that are extremely convenient for performing quantum mechanics calculations:
1. Integrals involving GTOs can usually be performed analytically, which is much more efficient and precise than numerical integration. This enables efficient construction of the electronic Hamiltonian matrix, since matrix elements like the kinetic energy $-\frac{1}{2} \langle \chi_\mu | \nabla^2 | \chi_\nu \rangle$ and electron-nuclear interaction $-Z_A \langle \chi_\mu | \,|\mathbf{r} - \mathbf{R}_A|^{-1}\, | \chi_\nu \rangle$ can be computed efficiently.
2. GTOs are *localized* in the sense that a GTO decays very quickly as $\mathbf{r}$ moves away from the GTO's center. This allows them to efficiently represent atomic orbitals and also enables efficient scaling for large calculations, as orbital pairs with negligible overlap can be ignored when constructing some parts of the Hamiltonian.

Using these GTOs, the atomic orbitals $\chi_\mu(\mathbf{r})$ can be constructed as
\begin{equation}
\chi_\mu(\mathbf{r}) = \sum_{\alpha} d_{\alpha,n,l} g_{\alpha lm}(\mathbf{r}-\mathbf{R}_A)
\end{equation}
In this case $\mu$ is a combined index representing the atom index $A$, orbital energy index $n$, and angular momentum numbers $l,m$. $\mathbf{R}_A$ is the atomic coordinate of atom $A$. $d_{\alpha,n,l}$ is an index representing the weight (coefficient) for each GTO in constructing $\chi_\mu(\mathbf{r})$.

### Basis Set Error

A finite set of functions $\{\chi_\mu(\mathbf{r})\}$ cannot represent the ground-state wave functions perfectly, and the resulting error in the ground-state energy and properties of a molecule is called the **basis set error** or **basis set incompleteness error**. The way to decrease the basis set error is to use more basis functions, which increases the computational cost of a calculation. For GTOs, the basis set size is typically increased by the **split-valence** constructions, in which the valence orbitals are split into multiple pieces to give the wave function more flexibility to change its shape in response to the electronic environment, thereby decreasing basis set error. Because the symbol $\zeta$ (zeta) historically referred to the exponents of the Gaussians used to construct GTOs, these larger basis sets are called $n$-zeta basis sets. For example, a double-zeta basis set has two basis functions per valence orbital, a triple-zeta basis has three, and so on. When naming basis sets, split-valence sets are typically denoted `dz` for double-zeta, `tz` for triple-zeta, and so on. When high accuracy is required or intermolecular (van der Waals) interactions are involved, the basis set must also be *augmented* with *diffuse* basis functions (i.e. broad basis functions with a small exponent $\alpha$ in the Gaussian), often indicated by an `aug` or `d` in the basis set name. You will see some examples of basis set names in the exercises below.

### Wave Function Methods

Quantum chemistry involves a variety of methods, and many of these methods explicitly describe the correlations between electrons. These correlated wave function approaches are typically implemented with single-particle basis sets; however, instead of constructing the wavefunction as a single Slater determinant of occupied orbitals (like in the Hartree-Fock method), they construct the wave function as a weighted sum of Slater determinants containing different orbitals, with the goal of capturing electronic correlations. These more advanced quantum chemistry methods also require basis sets and also suffer from basis set error that is mitigated by increasing the basis set size.

**SUMMARY**: For most quantum chemistry (and DFT) calculations on isolated molecules, the linear combination of atomic orbitals (LCAO) method is used to represent the wave functions. Gaussian-type orbitals (GTOs) are typically used to construct the atomic orbitals because they can be integrated analytically, leading to efficient code.

## Level of Theory

In quantum chemistry, except in rare cases for very small systems, exactly solving the many-body Schr&ouml;dinger equation is computationally infeasible, and approximate methods must be used. However, these approximations themselves have varying levels of accuracy and computational cost. More accurate methods are typically more computationally expensive, leading to a trade-off between accuracy and efficiency. The specific quantum mechanical method used to study a system is called the **level of theory**. A more accurate (more expensive) method is considered a *higher* level of theory.

An example of a low level of theory is semi-local DFT, which refers to DFT performed with semi-local exchange-correlation (XC) functionals like the LDA and GGAs. Another style of DFT, called hybrid DFT, is more expensive because it mixes a fraction of *exact exchange* into the XC functional, based on the exchange terms in the Hartree-Fock equations. This typically improves some properties like molecular bond energies by removing some *self-interaction error*. Self-interaction error occurs when an electron in a system spuriously repels itself in mean-field approximations. Due to its cost and improved accuracy for most systems, hybrid DFT is considered a higher level of theory than semi-local DFT.

One high level of theory we will explore here is the coupled cluster singles and doubles (CCSD) method, and its extension with perturbative triples CCSD(T). This method handles electronic exchange effects exactly (like Hartree-Fock) and solves for the electronic correlations in an approximate but explicit and rigorous way. CCSD(T) is very computationally expensive and scales poorly with the molecular system size and basis set size, but it is typically much more accurate than either semi-local or hybrid DFT, and it is regarded as the "gold standard" among practical quantum chemical approximations.

**SUMMARY**: The level of theory of a calculation is the specific method used to solve for the properties of interest. A higher level of theory is typically more accurate in the sense that the predicted ground-state energy will be closer to the exact ground state energy than for a lower level of theory.

## Assessing a Method's Basis Set and Level of Theory Together

In the above two sections, we saw that our quantum chemistry calculations can have two sources of error: basis set error and error due to the approximations made by a given level of theory. To decrease the total error of the ground state energy compared to the exact solution of the many-body Schr&ouml;dinger equation, we typically need to increase both the basis set size and level of theory. If we approach the limit of an infinitely large (complete) basis set and use the full configuration interaction (FCI) method (which is the exact solution for a given basis set), we approach the exact solution of the many-electron Schr&ouml;dinger equation and therefore the exact ground-state energy of the system (within the Born-Oppenheimer approximation).


## Before Getting Started

Typically, when we study systems using DFT, we need to relax the ionic positions to obtain the ground state stucture for a given method. For simplicity, in this exercise, we will use pre-existing geometries computed at a reasonably accurate level of theory and not relax them separately for each method.

The goal of the following exercises is to introduce you to how to try out different basis sets and levels of theory in a quantum chemistry code, in this case PySCF. The next couple blocks of code import the modules you need and define the molecular structures used in this exercise, as well as convenience functions for building the structures in PySCF. The exercises for you to complete are below.

Note this command below installs PySCF in this Colab notebook environment
```
%pip install pyscf
```
If you instead prefer to run this notebook or similar Python code on the GCP VM
then remove the line above and run your scripts in the "pyscf" environment. Pyscf is not installed in the "base" environment. 

```
(base) bond@instance:~$ conda activate pyscf
(pyscf) bond@instance:~$ python
```

In [ ]:
%pip install pyscf

from pyscf import gto, scf, dft
from pyscf.cc import CCSD
import numpy as np

# unit conversion for converting Hartree to kcal/mol
KCALPERMOL_PER_HARTREE = 627.509608

In [ ]:
# This block defines a function to construct the relevant molecules for a given basis.

# These are the atomic structures of some small molecules:
h2_geom = """
 H         0.000000000000      0.000000000000      0.000000000000
 H         0.000000000000      0.000000000000      0.741891560000
"""
o2_geom = """
 O         0.000000000000      0.000000000000      0.000000000000
 O         0.000000000000      0.000000000000      1.207800000000
"""
h2o_geom = """
 O         0.000000000000      0.000000000000      0.000000000000
 H         0.000000000000      0.000000000000      0.957900000000
 H         0.928958889248      0.000000000000     -0.233683101845
"""
f2_geom = """
 F         0.000000000000      0.000000000000      0.000000000000
 F         0.000000000000      0.000000000000      1.412900000000
"""
hf_geom = """
 F         0.000000000000      0.000000000000      0.000000000000
 H         0.000000000000      0.000000000000      0.915769000000
"""

def get_rxn1_mols(basis):
    h2o = gto.M(atom=h2o_geom, basis=basis)
    # O2 is an open-shell system with spin=2
    o2 = gto.M(atom=o2_geom, basis=basis, spin=2)
    h2 = gto.M(atom=h2_geom, basis=basis)
    return h2o, o2, h2

def get_rxn2_mols(basis):
    hf = gto.M(atom=hf_geom, basis=basis)
    f2 = gto.M(atom=f2_geom, basis=basis)
    h2 = gto.M(atom=h2_geom, basis=basis)
    return hf, f2, h2

## Exercise 1: Write a Script to Compute Reaction Energies with Hartree-Fock

In this exercise, you will write code that computes the reaction energy for two reactions:
- Reaction 1: $\ce{H2 + \frac{1}{2}O2 -> H2O}$
- Reaction 2: $\ce{\frac{1}{2}H2 + \frac{1}{2}F2 -> HF}$

To do so, fill in the functions `get_hf_rxn1_energy` and `get_hf_rxn2_energy`, which should take in the name of the basis set `basis` and return the energy of Reactions 1 and 2, respectively, computed with Hartree-Fock for that basis set.

PySCF uses a simple Python interface to set up molecular systems and DFT calculations. For example, see `get_rxn1_mols` for how to set up simple `Mole` objects (which store molecules). Below is a demonstration of how to run basic Hartree-Fock calculations. More documentation is available here, in case you need it: https://pyscf.org/user.html

In [ ]:
####### HARTREE-FOCK EXAMPLE #######

# Initialize the molecules with the basis set you want to use,
# and the total spin of the molecules
h2 = gto.M(atom=h2_geom, basis='def2-tzvp', verbose=0) # H2 molecule
hatom = gto.M(atom='H', basis='def2-tzvp', spin=1, verbose=0) # H atom

# Initialize Restricted Hartree-Fock (no spin polarization).
h2_calc = scf.RHF(h2)
# The kernel function runs the DFT calculation and returns the total energy.
e_h2 = h2_calc.kernel()

# If your molecule is open-shell, you need to use unrestricted Hartree-Fock.
# I.e. use scf.UHF instead of scf.RHF
h_calc = scf.UHF(hatom)
e_hatom = h_calc.kernel()

# Now we can calculate the atomization energy of the H2 molecule,
# i.e. the energy of the reaction H2 -> 2H
erxn = 2*e_hatom - e_h2

# PySCF uses atomic units, i.e. the units of energy are "Hartree".
# To convert to kcal/mol, multiply by the unit conversion factor provided:
print('Atomization energy of H2 with Hartree-Fock, in kcal/mol:',
      KCALPERMOL_PER_HARTREE*erxn)
# In case you need them, other unit conversions are available here:
# https://www.colby.edu/chemistry/PChem/Hartree.html

Now fill in the functions below to calculate the energy of Reactions 1 and 2:

In [ ]:
def get_hf_rxn1_energy(basis):
    h2o, o2, h2 = get_rxn1_mols(basis)
    ### YOUR CODE HERE ###
    return erxn

def get_hf_rxn2_energy(basis):
    hf, f2, h2 = get_rxn2_mols(basis)
    ### YOUR CODE HERE ###
    return erxn

Run the test below to make sure your routines work as expected. If you get an `AssertionError`, your code is not computing the reaction energy correctly!

In [ ]:
# This test calculates the energies of Reactions 1 and 2,
# using Hartree-Fock and def2-svp basis.

etest1 = get_hf_rxn1_energy('def2-svp')
etest2 = get_hf_rxn2_energy('def2-svp')
assert abs(-0.08688968393059393 - etest1) < 1e-8
assert abs(-0.11475360705714654 - etest2) < 1e-8

## Exercise 2: Test the Basis Set Convergence of Reactions 1 and 2 with Hartree-Fock

Using the function you wrote above, compute the energy of Reaction 1 with the following list of basis sets:

`['cc-pvdz', 'cc-pvtz', 'cc-pvqz', 'aug-cc-pvdz', 'aug-cc-pvtz', 'aug-cc-pvqz', 'def2-tzvppd']`

In the above basis set names that contain `cc`, the second to last letter indicates the polarization level:
- `pvdz` means double zeta
- `pvtz` means triple zeta
- `pvqz` means quadruple zeta

The prefix `aug` indicates that additional "diffuse" (i.e. slow-decaying) basis functions are added to the basis set. `def2-tzvppd` follows a different naming scheme because it was developed by different researchers with slightly different design choices, but you can see the similarities in naming convention: The `tzvp` represents that the basis is triple-zeta valence polarized, the second `p` indicates that extra polarization functions (higher-order spherical harmonics) are included, and the `d` stands for including diffuse functions (similar to the `aug` prefix in the `cc` basis sets). It is included to get you comfortable with different naming schemes, as you might need to use different classes of basis set in different applications.

**Assuming `aug-cc-pvqz` is a "complete" basis set (CBS), compute the basis set error for the other basis sets in kcal/mol**. Note: It is reasonable to expect `aug-cc-pvqz` to be close to the complete basis set in most cases, as it is quadruple-polarized and augmented with diffuse functions. We could get even closer to the CBS limit using `aug-cc-pv5z`, but this is quite expensive.

Definition of basis set error: For a method M and basis set B, the basis set error of B is the difference between the value of a property (in this case, the energy of Reaction 1) computed with method M and basis B and the value of that property computed with method M in the complete basis set limit.

In [ ]:
# List of basis sets, for convenience
basis_sets = ['cc-pvdz', 'cc-pvtz', 'cc-pvqz', 'aug-cc-pvdz', 'aug-cc-pvtz',
              'aug-cc-pvqz', 'def2-tzvppd']
complete_basis = 'aug-cc-pvqz'

### YOUR CODE HERE ###

print()
print("Energies in kcal/mol:", energies_kcal)
print("Basis Set Errors in kcal/mol:", errors_kcal)

Which basis sets have a basis set error of <1 kcal/mol for this reaction? Do diffuse functions improve the accuracy? What level of valence polarization is necessary to achieve this accuracy?

**YOUR ANSWER HERE**

## Exercise 3: RHF vs UHF for Dissociating the Hydrogen Molecule

The choice between using a restricted or unrestricted method can have significant impacts on the outcome of a calculation. Unrestricted calculations typically are twice as computationally expensive (and sometimes more, since convergence sometimes takes more steps). However, using a restricted calculation when the true ground-state of a system is spin-polarized can result in severe inaccuracies in the predicted properties. We will explore this concept by computing the dissociation curve of the hydrogen molecule ($\ce{H2}$) using restricted Hartree-Fock (RHF) and unrestricted Hartree-Fock (UHF).

**Important Note**: Sometimes, even when using UHF, a system that should be spin-polarized might converge to a non-spin-polarized state because it is stuck in a metastable electronic configuration or saddle point. To avoid this problem, it is necessary to provide a good **initial guess** density matrix to the UHF calculation. For this problem, we will use the density matrix of two far-apart hydrogen atoms as the initial guess for UHF. The method for doing this is demonstrated below.

In [ ]:
import matplotlib.pyplot as plt

def get_h2(bondlength, basis, verbose=2):
    # Helper function to make an H2 molecule with a given
    # bond length and basis.
    return gto.M(atom='H 0.0 0.0 0.0; H 0.0 0.0 {}'.format(bondlength),
                 basis=basis, verbose=verbose)

basis = 'def2-tzvp'

# The following few lines of code compute the density matrix
# of two far-apart H atoms, which will be used as an initial
# guess for the UHF calculations you do.
test_mol = get_h2(15.0, basis, verbose=3)
mf = scf.UHF(test_mol)
mf.kernel()
# Density matrix for two H atoms separated by 15 Angstroms
uhf_dm0 = mf.make_rdm1()

# Here is an example of how to use uhf_dm0 as an initial
# guess to a new calculation
new_mol = get_h2(3.0, basis, verbose=3)
mf = scf.UHF(new_mol)
# dm0 is the initial guess density matrix.
mf.kernel(dm0=uhf_dm0)

In the code block below, write your own code for the RHF and UHF dissociation curves of $\ce{H2}$, to answer the questions below.

In [ ]:
### YOUR CODE HERE ###

Plot the energy of the $\ce{H2}$ molecule with a bond length ranging from 0.6 to 5.0 Å, using RHF and UHF. Do the two approaches agree near the equilibrium bond length? Do they agree in the dissociation limit? Why do you think this is?

**YOU ANSWER HERE**

## Exercise 4: CCSD(T) and Basis Set Dependence

Coupled-cluster theory, especially CCSD and CCSD(T), are popular quantum chemistry methods. CCSD(T)&mdash;in which the coupled cluster equations are solved with singles and doubles excitations, followed by a perturbation theory-based correction for triples excitations&mdash;is often considered a "gold standard" for molecular chemistry problems (though it also has its limitations). The script below calculates the energy of Reaction 1 using CCSD(T) in two different basis sets. Read the code and comments to understand the steps used to perform the calculation, and compare the resulting predictions to the "near-exact" energy of Reaction 1 listed above. Then answer the questions below.

In [ ]:
def run_ccsdt_calc(mol, basis):
    # CCSD requires a baseline SCF calculation (Hartree-Fock or DFT)
    # to generate the molecular orbitals. Hartree-Fock is the most
    # popular baseline method, so we use it here.
    if mol.spin == 0:
        mf = scf.RHF(mol)
    else:
        mf = scf.UHF(mol)
    mf.kernel()
    # Pass the completed Hartree-Fock calculation to CCSD
    # and run the CCSD calculation.
    mycc = CCSD(mf)
    mycc.kernel()
    # Compute the perturbative triples correction.
    et = mycc.ccsd_t()
    # Sum the CCSD energy and triples correction, then
    # add it to the reaction energy.
    return mycc.e_tot + et

def get_ccsdt_reaction_energy(basis):
    # Load the molecules
    h2o, o2, h2 = get_rxn1_mols(basis)
    # Define the stoichiometry of the reaction
    # i.e. 1/2 O2 + H2 -> H2O
    counts = [1, -0.5, -1]
    e_h2o = run_ccsdt_calc(h2o, basis)
    e_o2 = run_ccsdt_calc(o2, basis)
    e_h2 = run_ccsdt_calc(h2, basis)
    return e_h2o - 0.5 * e_o2 - e_h2

eccsd1 = KCALPERMOL_PER_HARTREE * get_ccsdt_reaction_energy('cc-pvdz')
eccsd2 = KCALPERMOL_PER_HARTREE * get_ccsdt_reaction_energy('aug-cc-pvtz')

print()
print('CCSD(T) energy with cc-pvdz basis:', eccsd1, 'kcal/mol')
print('CCSD(T) energy with aug-cc-pvtz basis:', eccsd2, 'kcal/mol')

While standards vary widely based on the systems under consideration and the needs of a particular project, an error of <1 kcal/mol is typically considered quite good accuracy for predicting a molecular reaction energy. Using the reference energy for Reaction 1 provided below in Exercise 6, does CCSD(T) predict the Reaction 1 energy with <1 kcal/mol error with the `cc-pvdz` basis? With the `aug-cc-pvtz` basis?

**YOUR ANSWER HERE**

Your friend wants to calculate the energy of a particular reaction using quantum chemistry. They want to use CCSD(T) because it is more accurate than DFT. However, because the molecules involved contain a few dozen atoms, they think the CCSD(T) calculation is too expensive to do with a large basis set like `aug-cc-pvtz` or `def2-tzvppd`. Instead, they plan to use a smaller basis set, specifically `cc-pvdz`. Based on the accuracy with which CCSD(T) predicts the Reaction 1 energy for different basis sets, what do you think of this plan? What recommendations would you make regarding your friend's proposed methodology?

**YOU ANSWER HERE**

## Exercise 5: Compute DFT Reaction Energies

Setting up a DFT calculation is very similar to setting up a Hartree-Fock calculation, except that we also need to select an **exchange-correlation functional** to approximate the effects electron exchange and correlation on the system.

Read the example code and comments below to familiarize yourself with setting up a DFT calculation in PySCF. Then, write code to compute the reaction energies of Reactions 1 and 2 from Exercise 1.

To do so, fill in the functions `get_dft_rxn1_energy` and `get_dft_rxn2_energy`, which should take in the name of the exchange-correlation functional `xc` and the basis set `basis` and return the reaction energy, computed with DFT for that functional and basis set.

In [ ]:
####### DFT EXAMPLE #######

h2 = gto.M(atom=h2_geom, basis='def2-tzvp', verbose=0) # H2 molecule
hatom = gto.M(atom='H', basis='def2-tzvp', spin=1, verbose=0) # H atom

# Initialize Restricted DFT (no spin polarization).
h2_calc = dft.RKS(h2)
# Set the XC functional to PBE.
h2_calc.xc = 'PBE'
# The kernel function runs the DFT calculation and returns the total energy.
e_pbe = h2_calc.kernel()

# You can change the functional and rerun the calculation.
# In this case, we try out SCAN, which is a popular meta-GGA.
h2_calc.xc = 'SCAN'
e_scan = h2_calc.kernel()

# Different functionals give different ground-state energies!
print('Energy of H2 with PBE:', e_pbe)
print('Energy of H2 with SCAN:', e_scan)

# Just like for Hartree-Fock, you need to use unrestricted DFT
# for open-shell molecules. I.e. use dft.UKS instead of dft.RKS
h_calc = dft.UKS(hatom)
# For a given reaction, use the same functional and basis set for all molecules.
h_calc.xc = 'SCAN'
e_hatom = h_calc.kernel()

# Now we can calculate the atomization energy of the H2 molecule,
# i.e. the energy of the reaction H2 -> 2H
# Note: This is the atomization energy using the SCAN functional.
erxn = 2*e_hatom - e_scan
print('Atomization energy of H2 with SCAN, in kcal/mol:',
      KCALPERMOL_PER_HARTREE*erxn)

Now fill in the functions below to calculate the energy of Reactions 1 and 2:

In [ ]:
def get_dft_rxn1_energy(xc, basis):
    h2o, o2, h2 = get_rxn1_mols(basis)
    ### YOUR CODE HERE ###
    return erxn

def get_dft_rxn2_energy(xc, basis):
    hf, f2, h2 = get_rxn2_mols(basis)
    ### YOUR CODE HERE ###
    return erxn

Run the test below to make sure your routines work as expected. If you get an `AssertionError`, your code is not computing the reaction energy correctly!

In [ ]:
# This test calculates the energies of Reactions 1 and 2,
# using the PBE functional and def2-svp basis.

etest1 = get_dft_rxn1_energy('PBE', 'def2-svp')
etest2 = get_dft_rxn2_energy('PBE', 'def2-svp')
assert abs(-0.07907687493034765 - etest1) < 1e-8
assert abs(-0.08961561084256242 - etest2) < 1e-8

## Exercise 6: Test the Accuracy of Hybrid DFT vs Semi-local DFT vs Hartree-Fock

Choosing the right functional for a given application is just as important as choosing the right basis set. In this exercise, we will compare the accuracy of PBE (a semi-local functional), PBE0 (a hybrid functional), and the Hartree-Fock method. To assess the accuracy of the functional, we need two things:
1. We need to use a basis set with a very small basis set error, so that we can isolate the error of the functional itself. For this exercise, use `def2-tzvppd`, which is a triple-zeta polarized basis set with diffuse functions. You should have found that its basis set error in the previous problem was quite small (\<0.5 kcal/mol). The combination of triple-zeta polarization and diffuse functions make its typical basis set error similar to `aug-cc-pvtz` (though it will not behave identically in every case).
2. We need to know the "exact" ground-state energies of the systems of interest. [Karton *et al.*](https://www.sciencedirect.com/science/article/abs/pii/S0009261411005616) contains highly accurate theoretical reference values for various atomization energies, from which the energies of Reactions 1 and 2 can be derived. While these energies are still calculated with an approximate method, the method's expected error is so small that we can treat the resulting reaction energies as nearly exact. These reference energies are provided below.

Based on Karton *et al.*:
- Reaction 1 Reference Energy: -63.07 kcal/mol
- Reaction 2 Reference Energy: -67.37 kcal/mol

**Use the functions you wrote to compute the energies of Reactions 1 and 2 with the `def2-tzvppd` basis using Hartree-Fock, DFT with the PBE functional, and DFT with the PBE0 functional. Make sure to convert to kcal/mol.**

In [ ]:
# Compute and print the reaction energies here

### YOUR CODE HERE ###

print()
print('Reaction 1 Energy:')
print('HF: {:.2f} kcal/mol'.format(e1_hf))
print('PBE: {:.2f} kcal/mol'.format(e1_pbe))
print('PBE0: {:.2f} kcal/mol'.format(e1_pbe0))
print()
print('Reaction 2 Energy:')
print('HF: {:.2f} kcal/mol'.format(e2_hf))
print('PBE: {:.2f} kcal/mol'.format(e2_pbe))
print('PBE0: {:.2f} kcal/mol'.format(e2_pbe0))

You should find that DFT with PBE and PBE0 both reproduce the reaction energies better than Hartree-Fock. Why is this?

**YOUR ANSWER HERE**

Which functional more accurately predicts these two reaction energies? Note that PBE is a semi-local functional (specifically, a Generalized-Gradient Approximation or GGA), while PBE0 is a hybrid functional that uses 25% exact exchange and 75% PBE exchange. Based on this, can you explain why one functional predicts the reaction energies of these small molecules more accurately than the other?

**YOUR ANSWER HERE**